# 10K Analysis
In this demo, we explore answering compare an contrast queries with two different approaches:
1. ReAct agent, using prompting over a set of Tools
2. OpenAIAgent, using the function calling API to choose Tools

In [1]:
# # uncomment to turn on logging
# import logging
# import sys

# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
from llama_index import SimpleDirectoryReader, LLMPredictor, ServiceContext, GPTVectorStoreIndex
from llama_index.response.pprint_utils import pprint_response
from llama_index.llms import OpenAI

/Users/jerryliu/Programming/gpt_index/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=-1, streaming=True))
chatgpt = OpenAI(temperature=0, model="gpt-3.5-turbo")
gpt4 = OpenAI(temperature=0, model="gpt-4")
service_context = ServiceContext.from_defaults(llm=chatgpt)
service_context_gpt4 = ServiceContext.from_defaults(llm=gpt4)

## Load data

In [4]:
march_2022 = SimpleDirectoryReader(input_files=["../data/10q/uber_10q_march_2022.pdf"]).load_data()
june_2022 = SimpleDirectoryReader(input_files=["../data/10q/uber_10q_june_2022.pdf"]).load_data()
sept_2022 = SimpleDirectoryReader(input_files=["../data/10q/uber_10q_sept_2022.pdf"]).load_data()

# Build indices

In [5]:
march_index = GPTVectorStoreIndex.from_documents(march_2022)
june_index = GPTVectorStoreIndex.from_documents(june_2022)
sept_index = GPTVectorStoreIndex.from_documents(sept_2022)

In [6]:
march_engine = march_index.as_query_engine(similarity_top_k=3, service_context=service_context)
june_engine = june_index.as_query_engine(similarity_top_k=3, service_context=service_context)
sept_engine = sept_index.as_query_engine(similarity_top_k=3, service_context=service_context)

In [7]:
# define agent LLM
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
llm_gpt4 = ChatOpenAI(temperature=0, model_name="gpt-4")

## Compare and Contrast - ReAct Agent (LangChain)

In [8]:
from llama_index.langchain_helpers.agents import IndexToolConfig, LlamaIndexTool, LlamaToolkit

uber_config_sept = IndexToolConfig(
    query_engine=sept_engine, 
    name=f"Uber 10Q September 2022",
    description=f"Provides information about Uber quarterly financials ending September 2022",
    tool_kwargs={"return_direct": False}
)
uber_config_june = IndexToolConfig(
    query_engine=june_engine, 
    name=f"Uber 10Q June 2022",
    description=f"Provides information about Uber quarterly financials ending June 2022",
    tool_kwargs={"return_direct": False}
)
uber_config_march = IndexToolConfig(
    query_engine=march_engine, 
    name=f"Uber 10Q March 2022",
    description=f"Provides information about Uber quarterly financials ending March 2022",
    tool_kwargs={"return_direct": False}
)

In [9]:
toolkit = LlamaToolkit(
    index_configs=[uber_config_sept, uber_config_june, uber_config_march],
)

In [28]:
from llama_index.langchain_helpers.agents import create_llama_agent

# initialize both gpt-3 and gpt-4 agent chains
agent_chain = create_llama_agent(
    toolkit,
    llm,
    # memory=memory,
    verbose=True
)

agent_chain_gpt4 = create_llama_agent(
    toolkit,
    llm_gpt4,
    verbose=True
)

In [23]:
# vague answer? 
agent_chain.run(input="Analyze Uber revenue growth over the last few quarters")



> Entering new AgentExecutor chain...
 I need to compare the revenue from different quarters
Action: Uber 10Q September 2022
Action Input: Revenue
Observation: increased in the nine months ended September 30, 2022 compared to the same period in 2021 primarily due to what?

The increase in revenue in the nine months ended September 30, 2022 compared to the same period in 2021 was primarily due to an increase in Gross Bookings of 31%, or 36% on a constant currency basis. The increase in Gross Bookings was primarily driven by increases in Mobility Trip volumes as the business recovers from the impacts of COVID-19 and a $4.4 billion increase in Freight Gross Bookings resulting primarily from the acquisition of Transplace in the fourth quarter of 2021. Additionally, during the first nine months of 2022, we saw a $2.2 billion net increase in Mobility revenue as a result of business model changes in the UK and an accrual made for the resolution of historical claims in the UK relating to the

'Uber revenue increased in the nine months ended September 30, 2022 compared to the same period in 2021 primarily due to an increase in Gross Bookings of 31%, or 36% on a constant currency basis, an increase in Mobility Trip volumes, the acquisition of Transplace, business model changes in the UK, and an increase in certain Courier payments and incentives.'

In [12]:
agent_chain_gpt4.run(input="Analyze Uber revenue growth over the last few quarters")



> Entering new AgentExecutor chain...
I need to check the revenue figures for the last few quarters.
Action: Uber 10Q September 2022
Action Input: Revenue for September 2022
Observation: 
$8,343 million
Thought:I have the revenue for September 2022, now I need the revenue for June 2022 and March 2022.
Action: Uber 10Q June 2022
Action Input: Revenue for June 2022
Observation: Six Months Ended June 30, 2022 Compared with the Same Period in 2021
Revenue increased $8.1 billion, or 118%, primarily attributable to an increase in Gross Bookings of 34%, or 37% on a constant currency basis.

Answer: $14.927 billion
Thought:I have the revenue for June 2022, now I need the revenue for March 2022.
Action: Uber 10Q March 2022
Action Input: Revenue for March 2022
Observation: 
Revenue for March 2022 was $6,854 million.
Thought:I now have the revenue figures for the last three quarters.
Final Answer: Uber's revenue growth over the last few quarters is as follows: March 2022 - $6,854 million, June 

"Uber's revenue growth over the last few quarters is as follows: March 2022 - $6,854 million, June 2022 - $14,927 billion, and September 2022 - $8,343 million."

In [25]:
# only picks september and june 
agent_chain.run(input="Analyze changes in risk factors for Uber")



> Entering new AgentExecutor chain...
 I need to compare the risk factors across different quarters
Action: Uber 10Q September 2022
Action Input: Risk Factors
Observation: 
The risk factors mentioned in the context information include: 
• Drivers being classified as employees, workers or quasi-employees instead of independent contractors. 
• High competition in the mobility, delivery, and logistics industries. 
• Lowering of fares or service fees and offering significant Driver incentives and consumer discounts and promotions. 
• Incurring significant losses since inception. 
• Drivers obtaining additional or alternative opportunities for work. 
• Increases in fuel, food, labor, energy, and other costs due to inflation and other factors. 
• Decline in consumer spending. 
• Poor economic conditions. 
• Dependencies on third parties. 
• Investment in climate commitments. 
• Outbreaks of contagious disease, such as the COVID-19 pandemic. 
• Occurrence of a catastrophic event.
Thought: I

'The risk factors for Uber have changed over the past three quarters, with the most recent quarter including risks related to data collection, use, transfer, disclosure, and other processing, intellectual property, stock price volatility, climate change risks, economic risks, risk of catastrophic events, health epidemics, and outbreaks of contagious disease.'

In [13]:
agent_chain_gpt4.run(input="Analyze changes in risk factors for Uber")



> Entering new AgentExecutor chain...
I should start by checking the most recent 10Q for any updates on risk factors.
Action: Uber 10Q September 2022
Action Input: Risk factors
Observation: 
•Risk of Drivers being classified as employees, workers or quasi-employees instead of independent contractors. 
•Highly competitive mobility, delivery, and logistics industries. 
•Lowering of fares or service fees and offering of Driver incentives and consumer discounts and promotions. 
•Significant losses since inception and potential for future losses. 
•Decrease in consumer spending and potential for merchants to go out of business. 
•Increases in fuel, food, labor, energy, and other costs due to inflation and other factors. 
•Dependencies on third parties. 
•Outbreaks of contagious disease, such as the COVID-19 pandemic. 
•Catastrophic events, such as disease, a weather event, war, or terrorist attack. 
•Failure to meet regulatory requirements related to climate change. 
•Impact of actions to

'Comparing the risk factors from the Uber 10Q September 2022 and Uber 10Q June 2022, some changes include:\n\n1. The September 2022 report highlights the risk of drivers being classified as employees, workers, or quasi-employees instead of independent contractors, which was not mentioned in the June 2022 report.\n2. The September 2022 report emphasizes the highly competitive nature of the mobility, delivery, and logistics industries, while the June 2022 report does not.\n3. The September 2022 report mentions the lowering of fares or service fees and offering of driver incentives and consumer discounts and promotions as a risk factor, which was not present in the June 2022 report.\n4. The September 2022 report highlights the significant losses since inception and potential for future losses, which was not mentioned in the June 2022 report.\n5. The September 2022 report includes the risk of a decrease in consumer spending and potential for merchants to go out of business, which was not p

In [26]:
# Prompt variation 1: 
agent_chain.run(input="Analyze Uber revenue growth and risk factors over time")



> Entering new AgentExecutor chain...
 I need to compare Uber's quarterly financials
Action: Uber 10Q September 2022
Action Input: Financials
Observation: 
The financial information provided in this Quarterly Report on Form 10-Q includes the Condensed Consolidated Balance Sheets as of December 31, 2021 and September 30, 2022, the Condensed Consolidated Statements of Operations for the Three and Nine Months Ended September 30, 2021 and 2022, the Condensed Consolidated Statements of Comprehensive Income (Loss) for the Three and Nine Months Ended September 30, 2021 and 2022, the Condensed Consolidated Statements of Redeemable Non-Controlling Interests and Equity for the Three and Nine Months Ended September 30, 2021 and 2022, and the Condensed Consolidated Statements of Cash Flows for the Nine Months Ended September 30, 2021 and 2022.
Thought: I need to compare the financials from the other quarters
Action: Uber 10Q June 2022
Action Input: Financials
Observation: 
The financial informat

"Uber's revenue growth and risk factors can be analyzed by comparing the financials from the quarterly reports ending September 2022, June 2022, and March 2022. This comparison will provide information about the company's net cash provided by (used in) operating activities, net cash used in investing activities, net cash used in financing activities, principal payments on finance leases, proceeds from issuance of common stock for the ESPP, and sources of funding and available line of credit."

In [19]:
# Prompt variation 2: looks at all three quarters
agent_chain.run(input="Analyze Uber revenue growth and risk factors over quarters")



> Entering new AgentExecutor chain...
 I need to compare Uber's revenue growth and risk factors over different quarters
Action: Uber 10Q September 2022
Action Input: Revenue growth and risk factors
Observation: 
Revenue growth may be affected by a number of factors, including our ability to attract and retain new platform users, increased competition in the markets in which we operate, our ability to expand our operations in new and existing markets, our ability to maintain an adequate growth rate and effectively manage that growth, our ability to keep pace with technological changes in the industries in which we operate, changes in governmental or other regulations affecting our business, harm to our brand or reputation, and other risks described elsewhere in this Quarterly Report on Form 10-Q. Seasonal trends may also contribute to fluctuations in our operating results. In addition, our revenue growth may be affected by a softening of Driver, consumer, merchant, Shipper, or Carrier

"Uber's revenue growth and risk factors vary from quarter to quarter and are affected by a number of factors, including the duration and severity of the COVID-19 pandemic, the ability to grow supply and demand on the platform, the ability to increase existing platform users' activity, the ability to introduce the platform to new markets, the ability to provide high-quality support to drivers, consumers, merchants, shippers, and carriers, the ability to expand the business and increase market share and category position, the ability to compete with the products and offerings of competitors, the ability to develop new products, offerings, and technologies, the ability to penetrate suburban and rural areas, the ability to reduce the costs of the Mobility offering, the ability to maintain existing local regulations in key markets, the ability to enter or expand operations in certain countries, and the ability to increase positive perception of the brand. Risk factors include the inability 

In [14]:
# Prompt variation 1: only looks at september
agent_chain_gpt4.run(input="Analyze Uber revenue growth and risk factors over time")



> Entering new AgentExecutor chain...
I need to gather revenue data and risk factors from each of the three available 10Q reports.
Action: Uber 10Q September 2022
Action Input: Revenue and risk factors
Observation: 
Revenue for the company may be affected by a variety of factors, including competition, consumer spending, and economic conditions. The company may experience significant fluctuations in its operating results due to seasonal fluctuations, the current COVID-19 pandemic, and other factors beyond its control. Risk factors that could have an adverse effect on the company's business, financial condition, operating results, or prospects include Drivers being classified as employees, workers or quasi-employees instead of independent contractors, increased competition, the need to lower fares or service fees, and the need to offer significant Driver incentives and consumer discounts and promotions. Additionally, the company may be adversely affected by geopolitical shifts, increa

"Uber's revenue growth is affected by various factors such as competition, consumer spending, and economic conditions. Risk factors include driver classification, increased competition, fare and fee reductions, and geopolitical shifts. The company's revenue in June 2022 was expected to be recognized as $33 million for contracts less than or equal to 12 months and $115 million for contracts greater than 12 months. In March 2022, revenue was generated from fees paid by drivers and merchants, as well as fees paid by end-users in certain markets."

In [14]:
# only picks september 
agent_chain.run(input="What have been the biggest changes in the macro environment in the past few quarters?")



> Entering new AgentExecutor chain...
 I need to look at the changes in the macro environment over the past few quarters.
Action: Uber 10Q September 2022
Action Input: Macro environment
Observation: risks that Uber faces include:

• Outbreaks of contagious disease, such as the COVID-19 pandemic and the impact of actions to mitigate the such disease or pandemic
• Catastrophic events, such as disease, a weather event, war, or terrorist attack
• Laws and regulations more restrictive than those in the United States
• Competition with companies or other services that understand local markets better than Uber does
• Differing levels of social acceptance of Uber's brand, products, and offerings
• Differing levels of technological compatibility with Uber's platform
• Legal uncertainty regarding Uber's liability for the actions of platform users and third parties
• Fluctuations in currency exchange rates
• Regulations governing the control of local currencies
• Adverse tax consequences
• Poli

"The biggest changes in the macro environment in the past few quarters include risks such as outbreaks of contagious disease, catastrophic events, laws and regulations more restrictive than those in the United States, competition with companies or other services that understand local markets better than Uber does, differing levels of social acceptance of Uber's brand, products, and offerings, differing levels of technological compatibility with Uber's platform, legal uncertainty regarding Uber's liability for the actions of platform users and third parties, fluctuations in currency exchange rates, regulations governing the control of local currencies, adverse tax consequences, political, social, and economic instability abroad, public health concerns or emergencies, reduced or varied protection for intellectual property rights in some markets, increased financial accounting and reporting burdens, import and export restrictions, and changes in trade regulation."

In [30]:
# get wrong answer?
agent_chain.run(input="How much cash did Uber have in sept 2022?")



> Entering new AgentExecutor chain...
 I need to look at the financials for September 2022
Action: Uber 10Q September 2022
Action Input: N/A
Observation: 
The question is not provided.
Thought: I need to look at the financials for June 2022 and March 2022 to get the information I need
Action: Uber 10Q June 2022
Action Input: N/A
Observation: 
The question is not provided.
Thought: I need to look at the financials for March 2022
Action: Uber 10Q March 2022
Action Input: N/A
Observation: 
N/A
Thought: I now know the final answer
Final Answer: Uber had $7.2 billion in cash as of September 2022.

> Finished chain.


'Uber had $7.2 billion in cash as of September 2022.'

In [17]:
# get better answer
agent_chain_gpt4.run(input="How much cash did Uber have in sept 2022?")



> Entering new AgentExecutor chain...
I need to check the Uber 10Q September 2022 to find the cash amount.
Action: Uber 10Q September 2022
Action Input: Cash amount in September 2022
Observation: 
$8.6 billion
Thought:I now know the final answer.
Final Answer: Uber had $8.6 billion in cash in September 2022.

> Finished chain.


'Uber had $8.6 billion in cash in September 2022.'

In [16]:
agent_chain.run(input="How much cash did Uber have in March 2022?")



> Entering new AgentExecutor chain...
 I need to find out how much cash Uber had in March 2022.
Action: Uber 10Q March 2022
Action Input: Cash
Observation: paid for income taxes, net of refunds, for the three months ended March 31, 2021 was:

22
Thought: I now know the final answer
Final Answer: Uber had $22 million in cash in March 2022.

> Finished chain.


'Uber had $22 million in cash in March 2022.'

In [ ]:
# get wrong answer?
agent_chain_gpt4.run(input="How much cash did Uber have in March 2022?")

#### Define Graph

In [37]:
from llama_index import ComposableGraph, GPTListIndex

In [38]:
graph = ComposableGraph.from_indices(
    GPTListIndex,
    children_indices=[march_index, june_index, sept_index],
    index_summaries=[
        "Provides information about Uber quarterly financials ending March 2022",
        "Provides information about Uber quarterly financials ending June 2022",
        "Provides information about Uber quarterly financials ending September 2022"
    ]
)

In [39]:
from llama_index.query_engine.transform_query_engine import TransformQueryEngine
from llama_index.indices.query.query_transform.base import DecomposeQueryTransform

# define decompose_transform
decompose_transform = DecomposeQueryTransform(verbose=True)

# define custom query engines
custom_query_engines = {}
for index in [march_index, june_index, sept_index]:
    query_engine = index.as_query_engine(service_context=service_context)
    query_engine = TransformQueryEngine(
        query_engine,
        query_transform=decompose_transform,
        transform_extra_info={'index_summary': index.index_struct.summary},
    )
    custom_query_engines[index.index_id] = query_engine

custom_query_engines[graph.root_id] = graph.root_index.as_query_engine(
    service_context=service_context,
    streaming=True,
)

In [40]:
# define graph
g_engine = graph.as_query_engine(
    custom_query_engines=custom_query_engines
)

In [119]:
# test querying the graph
response = g_engine.query('Analyze Uber revenue growth over the last few quarters')
print(str(response))

> Current query: Analyze Uber revenue growth over the last few quarters
> New query:  What was Uber's revenue growth from the last quarter ending March 2022 compared to the previous quarter?
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 19 tokens
> [retrieve] Total embedding token usage: 19 tokens
> [retrieve] Total embedding token usage: 19 tokens
> Current query: Analyze Uber revenue growth over the last few quarters
> New query:  What was Uber's revenue growth from the last quarter ending March 2022 compared to the previous quarter?
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1653 tokens
> [get_response] Total LLM token usage: 1653 tokens
> [get_response] Total LLM token usage: 1653 tokens
INFO:llama_index.token_counter.token

In [59]:
response.source_nodes[2]

NodeWithScore(node=Node(text="\nUber's revenue growth from the last few quarters ending September 2022 was 99%, with total revenue increasing from $11.7 billion to $23.3 billion.", doc_id='b65df649-59a3-4ca3-b57f-d6d4f5404364', embedding=None, doc_hash='6b64e2c2316f4da7944aeb5fff13ca96f08590d2ebcfa09732e68905983b7889', extra_info=None, node_info=None, relationships={}), score=None)

## Compare and Contrast - OpenAI Function Agent

Use OpenAIAgent, built on top of the OpenAI tool use interface

In [8]:
from llama_index.tools.query_engine import QueryEngineTool


query_tool_sept = QueryEngineTool.from_defaults(
    query_engine=sept_engine,
    name="sept_2022",
    description=f"Provides information about Uber quarterly financials ending September 2022",
)
query_tool_june = QueryEngineTool.from_defaults(
    query_engine=june_engine,
    name="june_2022",
    description=f"Provides information about Uber quarterly financials ending June 2022",
)
query_tool_march = QueryEngineTool.from_defaults(
    query_engine=march_engine,
    name="march_2022",
    description=f"Provides information about Uber quarterly financials ending March 2022",
)

In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
from llama_index.agent import OpenAIAgent
from llama_index.selectors.llm_selectors import LLMSingleSelector


query_engine = OpenAIAgent.from_tools(
    [
        query_tool_sept,
        query_tool_june,
        query_tool_march,
    ],
    max_function_calls=10,
    verbose=True
)

In [13]:
response = query_engine.query("Analyze Uber revenue growth over the last few quarters")

=== Calling Function ===
Calling function: sept_2022 with args: {
  "input": "revenue"
}
Got output: 
We generate revenue primarily from fees paid by Mobility Drivers for the use of our platform(s) and related services to facilitate and complete mobility services, and in certain markets, revenue from fees paid by end-users for connection services obtained via the platform. Additionally, in certain markets where we are responsible for delivery services, delivery fees charged to end-users are also included in revenue. We also generate revenue for Delivery from Merchants’ and Couriers’ use of the Delivery platform and related service to facilitate and complete Delivery transactions. Freight revenue consists of revenue from freight transportation services provided to Shippers. Prior to 2022, All Other revenue primarily includes collaboration revenue related to our Advanced Technologies Group (“ATG”) business and revenue from our New Mobility offerings and products.
=== Calling Function ===

In [14]:
print(str(response))

Based on the information provided, Uber's revenue growth over the last few quarters can be attributed to the following factors:

1. Fees paid by Mobility Drivers: Revenue is primarily generated from fees paid by Mobility Drivers for the use of Uber's platform and related services to facilitate and complete mobility services. This includes fees paid by Drivers for connecting with passengers and providing transportation services.

2. Fees paid by end-users: In certain markets, Uber also generates revenue from fees paid by end-users for connection services obtained via the platform. This includes fees paid by passengers for booking rides and accessing transportation services.

3. Delivery revenue: Uber's revenue also includes fees paid by Merchants and Couriers for the use of the Delivery platform and related services to facilitate and complete delivery transactions. In certain markets where Uber is responsible for delivery services, delivery fees charged to end-users are also included in

In [26]:
response = query_engine.query("Analyze changes in risk factors for Uber")

In [27]:
print(str(response))

To analyze changes in risk factors for Uber, we would need access to the company's financial reports and disclosures. These reports typically provide information about the company's risk factors and any changes that have occurred over time.

Unfortunately, as an AI language model, I don't have real-time access to current financial reports. However, I can provide you with general information about common risk factors that may affect Uber or any other company in the transportation industry. These risk factors can include:

1. Regulatory and Legal Risks: Uber operates in multiple countries and is subject to various regulations and legal challenges. Changes in regulations or legal actions can impact Uber's operations and financial performance.

2. Competitive Landscape: Uber faces competition from other ride-sharing companies, taxi services, and transportation network companies. Changes in the competitive landscape, such as the entry of new competitors or changes in market share, can affec

In [21]:
response = query_engine.query("Analyze Uber revenue growth and risk factors over quarters")

=== Calling Function ===
Calling function: sept_2022 with args: {
  "input": "revenue"
}
Got output: What is the primary source of revenue for Uber?

The primary source of revenue for Uber is fees paid by Mobility Drivers and Merchants for the use of their platform(s) and related services to facilitate and complete mobility and delivery services. In certain markets, revenue from fees paid by end-users for connection services obtained via the platform is also included in revenue. Additionally, revenue from advertising services and freight transportation services provided to shippers are also included in revenue.
=== Calling Function ===
Calling function: june_2022 with args: {
  "input": "revenue"
}
Got output: What is the primary source of revenue for Uber?

The primary source of revenue for Uber is fees paid by Mobility Drivers and Merchants for the use of their platform(s) and related services to facilitate and complete mobility and delivery services. In certain markets, revenue from

In [22]:
print(str(response))

Based on the available information, the primary source of revenue for Uber is fees paid by Mobility Drivers and Merchants for the use of their platform and related services. This includes fees paid by end-users for connection services obtained via the platform, revenue from advertising services, and revenue from freight transportation services provided to shippers.

As for the risk factors, some of the key risks mentioned include the impact of the COVID-19 pandemic on Uber's business and financial results, economic conditions affecting consumer spending, climate change risks, security and data privacy breaches, the need for additional capital, legal and regulatory risks, and risks related to data collection and processing.

Please note that the specific details and analysis of revenue growth and risk factors may vary in the actual financial reports and disclosures provided by Uber.


In [32]:
response.source_nodes[2]

NodeWithScore(node=Node(text="The context information provided does not include data for September 2019, so it is not possible to determine Uber's revenue growth from September 2019 to September 2022.", doc_id='482b9a72-cc11-46a2-833d-99d5a24f71bc', embedding=None, doc_hash='cd1c6c860a4bd10f1216afef4b4ccdc4e4966d7b131659bd072af03f15cf493e', extra_info=None, node_info=None, relationships={}), score=None)

In [29]:
response = query_engine.query("How much cash did Uber have in sept 2022?")

=== Calling Function ===
Calling function: sept_2022 with args: {
  "input": "cash"
}
Got output: What is the amount of net cash provided by operating activities for the nine months ended September 30, 2022?

The amount of net cash provided by operating activities for the nine months ended September 30, 2022 is $886 million.


In [30]:
print(str(response))

Uber had $886 million in net cash provided by operating activities for the nine months ended September 30, 2022.


In [32]:
response.source_nodes

[]